<a href="https://colab.research.google.com/github/alexanderthclark/Intro-Stats-2023-Fall/blob/main/Interactive/p-values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate a P-value

Go to File -> Save a copy to make your own private copy (might be necessary for permissions).

In [ ]:
#@title Input
import ipywidgets as widgets
from IPython.display import display, HTML
from IPython.display import Markdown as md
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np

alternative = widgets.Dropdown(description = 'Alternative',
                               options = ['Two-sided', '>', '<'])

statistic = widgets.FloatText(description = 'Statistic',
                              value=0,
                              disabled=False)

stat_type = widgets.Dropdown(description = 'Kind',
                             value = 'z',
                             options = ['t', 'z', 'chi2'])

stat_input = widgets.HBox([statistic, stat_type])

degrees_of_freedom = widgets.IntText(description = 'Deg Freedom',
                                     value = 10,)

calc_button = widgets.Button(description = 'Get P-value')
calc_output = widgets.Output()
plot_output = widgets.Output()

def get_pvalue():

  global result
  if stat_type.value == 'z':

    if alternative.value == 'Two-sided':
      result = 2*(1 - stats.norm.cdf(statistic.value))
    elif alternative.value == '>':
      result = (1 - stats.norm.cdf(statistic.value))
    else:
      result = stats.norm.cdf(statistic.value)

  elif stat_type.value == 't':

    dof = degrees_of_freedom.value
    if alternative.value == 'Two-sided':
      result = 2*(1 - stats.t.cdf(x = statistic.value, df = dof))
    elif alternative.value == '>':
      result = (1 - stats.t.cdf(x=statistic.value, df = dof))
    else:
      result = stats.t.cdf(x = statistic.value, df = dof)

  elif stat_type.value == 'chi2':
    dof = degrees_of_freedom.value
    result = 1 - stats.chi2.cdf(x = statistic.value, df = dof)


  # show user the result
  calc_output.clear_output()
  with calc_output:
    s = "<tt>The P-value is **{:,.3f}**.</tt>".format(result)
    display(md(s))



  # make plot

  xmax = np.max([np.abs(statistic.value)*1.1, 4])
  x = np.linspace(-xmax, xmax, 100_000)

  if stat_type.value == 'z':
    y = stats.norm.pdf(x)
  elif stat_type.value == 't':
    y = stats.t.pdf(x = x, df = dof)
  elif stat_type.value == 'chi2':
    xmax = np.max([np.abs(statistic.value)*2, 4])
    x = np.linspace(0, xmax, 100_000)
    y = stats.chi2.pdf(x = x, df = dof)

  plot_output.clear_output()
  with plot_output:

    fig, ax = plt.subplots(figsize = (14,4))
    ax.plot(x,y, lw = 2)

    # add z stat
    ax.axvline(statistic.value, linestyle = 'dashed')

    # shade p-value
    if stat_type.value != 'chi2':
      if alternative.value == 'Two-sided':
        ax.axvline(-statistic.value, linestyle = 'dashed', alpha = 0.4)

        right_x = x[x > np.abs(statistic.value)]
        right_y = y[-len(right_x):]
        ax.fill_between(right_x, right_y, alpha = .2, color = 'C0')

        left_x = x[x < -np.abs(statistic.value)]
        left_y = y[:len(left_x)]
        ax.fill_between(left_x, left_y, alpha = .2, color = 'C0')

      elif alternative.value == '>':
        right_x = x[x > statistic.value]
        right_y = y[-len(right_x):]
        ax.fill_between(right_x, right_y, alpha = .2, color = 'C0')

      else:
        left_x = x[x < statistic.value]
        left_y = y[:len(left_x)]
        ax.fill_between(left_x, left_y, alpha = .2, color = 'C0')
    else:
      ax.axvline(statistic.value, linestyle = 'dashed', alpha = 0.4)
      right_x = x[x > np.abs(statistic.value)]
      right_y = y[-len(right_x):]
      ax.fill_between(right_x, right_y, alpha = .2, color = 'C0')


    # clean plot
    for s in 'left', 'right', 'top':
      ax.spines[s].set_visible(False)
    ax.yaxis.set_ticks([])
    ylim = ax.get_ylim()
    ax.set_ylim(0, ylim[1])
    plt.show(block = True)

display(alternative)
display(stat_input)
display(degrees_of_freedom)
display(calc_button)
display(calc_output)
display(plot_output)
calc_button.on_click(lambda x: get_pvalue())


Dropdown(description='Alternative', options=('Two-sided', '>', '<'), value='Two-sided')

IntText(value=10, description='Deg Freedom')

Button(description='Get P-value', style=ButtonStyle())

Output()

Output()